In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-ham-emails/emails.csv


In [2]:
emails = pd.read_csv('../input/spam-ham-emails/emails.csv')

In [3]:
emails

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


In [4]:
emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5728 non-null   object
 1   spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB


There are 

In [5]:
sw = ("i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "would", "should", "could", "ought", "i'm", "you're", "he's", "she's", "it's", "we're", "they're", "i've", "you've", "we've", "they've", "i'd", "you'd", "he'd", "she'd", "we'd", "they'd", "i'll", "you'll", "he'll", "she'll", "we'll", "they'll", "isn't", "aren't", "wasn't", "weren't", "hasn't", "haven't", "hadn't", "doesn't", "don't", "didn't", "won't", "wouldn't", "shan't", "shouldn't", "can't", "cannot", "couldn't", "mustn't", "let's", "that's", "who's", "what's", "here's", "there's", "when's", "where's", "why's", "how's", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very")

In [6]:
emails['spam'].value_counts()

0    4360
1    1368
Name: spam, dtype: int64

In [7]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
import string 
from nltk.corpus import stopwords
from nltk import PorterStemmer as Stemmer
def preprocess(email):
    email = email.lower()
    email = ''.join([t for t in email if t not in string.punctuation])
    email = [t for t in email.split() if  t not in stopwords.words('english')]
    st = Stemmer()
    email =[st.stem(t) for t in email]
    email = ' '.join(email)
    return email

How many of the emails are spam? 

There are 1368 spam emails.

In [9]:
first_w = emails.apply(lambda x: x['text'].split()[0],axis=1)
first_w.value_counts()

Subject:    5728
dtype: int64

Which word appears at the beginning of every email in the dataset? Respond as a lower-case word with punctuation removed.

subject


Could a spam classifier potentially benefit from including the frequency of the word that appears in every email?

Yes -- the number of times the word appears might help us differentiate spam from ham.

No -- the word appears in every email so this variable would not help us differentiate spam from ham.

Yes

In [10]:
characters = emails.apply(lambda x: len(x['text']),axis=1)
max(characters)

43952

The nchar() function counts the number of characters in a piece of text. How many characters are in the longest email in the dataset (where longest is measured in terms of the maximum number of characters)?

43952

In [11]:
corpus = emails.apply(lambda x:preprocess(x['text']),axis=1)

In [12]:
corpus

0       subject natur irresist corpor ident lt realli ...
1       subject stock trade gunsling fanni merril muzo...
2       subject unbeliev new home made easi im want sh...
3       subject 4 color print special request addit in...
4       subject money get softwar cd softwar compat gr...
                              ...                        
5723    subject research develop charg gpg forward shi...
5724    subject receipt visit jim thank invit visit ls...
5725    subject enron case studi updat wow day super t...
5726    subject interest david pleas call shirley cren...
5727    subject news aurora 5 2 updat aurora version 5...
Length: 5728, dtype: object

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
vec = CountVectorizer()
X = vec.fit_transform(corpus)

In [15]:
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
df.head()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,0000,000000,00000000,0000000000,000000000003619,000000000003991,000000000003997,000000000005168,...,zwzm,zxghlajf,zyban,zyc,zygoma,zymg,zzmacmac,zzn,zzncacst,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
(df.sum() > 286).sum()

585

In [17]:
df.to_csv('./dtm.csv',index=False)

In [18]:
vec = CountVectorizer(min_df =0.05)
X = vec.fit_transform(corpus)

In [19]:
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
df.head()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,01,02,03,04,05,06,07,08,...,week,well,wish,within,without,work,would,write,www,year
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(X)
X_tfidf = tfidf_transformer.transform(X)
print(X_tfidf.shape)

(5728, 366)


In [21]:
print(X_tfidf)

  (0, 360)	0.1415455597275026
  (0, 359)	0.1250153838595609
  (0, 354)	0.26946806358529657
  (0, 345)	0.09006579980716196
  (0, 321)	0.11772824363289099
  (0, 316)	0.036079141087253803
  (0, 309)	0.1390563561831101
  (0, 295)	0.09320765209166625
  (0, 283)	0.12770623631265213
  (0, 269)	0.1380855583929135
  (0, 263)	0.21086657306659284
  (0, 260)	0.1167135441683233
  (0, 241)	0.11760737537550232
  (0, 240)	0.11754709275314507
  (0, 224)	0.2275518091290443
  (0, 209)	0.4034776834841858
  (0, 206)	0.09268983366452667
  (0, 205)	0.18352858795979157
  (0, 203)	0.12810756571315235
  (0, 201)	0.08759347276159778
  (0, 197)	0.10573592939548278
  (0, 176)	0.09445618986125946
  (0, 175)	0.08570086564377546
  (0, 153)	0.20791403977704218
  (0, 144)	0.13551839183773817
  :	:
  (5727, 206)	0.046288429009372654
  (5727, 205)	0.04582622321728491
  (5727, 203)	0.0639757104596707
  (5727, 175)	0.21399102137518883
  (5727, 172)	0.10849501848170634
  (5727, 155)	0.04494994646508352
  (5727, 140)	0.06409

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, emails['spam'], test_size=0.3, random_state=123)

In [23]:
from sklearn.tree import DecisionTreeClassifier
spamCART = DecisionTreeClassifier(random_state=123)
spamCART.fit(X_train,y_train)
spamCART.score(X_test,y_test)

0.9552065154159395

In [24]:
from sklearn.metrics import classification_report
y_predTrainCART = spamCART.predict(X_train)
print(classification_report(y_train, y_predTrainCART))
spamCART.score(X_train,y_train)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3041
           1       1.00      1.00      1.00       968

    accuracy                           1.00      4009
   macro avg       1.00      1.00      1.00      4009
weighted avg       1.00      1.00      1.00      4009



0.9992516837116487

In [25]:
from sklearn.metrics import classification_report
y_predCART = spamCART.predict(X_test)
print(classification_report(y_test, y_predCART))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1319
           1       0.90      0.91      0.90       400

    accuracy                           0.96      1719
   macro avg       0.94      0.94      0.94      1719
weighted avg       0.96      0.96      0.96      1719



In [26]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predCART)
metrics.auc(fpr, tpr)

0.9377160727824109

In [27]:
from sklearn.ensemble import RandomForestClassifier
spamRF= RandomForestClassifier(random_state=123)
spamRF.fit(X_train,y_train)
spamRF.score(X_test,y_test)

0.9761489237929029

In [28]:
from sklearn.metrics import classification_report
y_predTrainRF = spamRF.predict(X_train)
print(classification_report(y_train, y_predTrainRF))
spamRF.score(X_train,y_train)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3041
           1       1.00      1.00      1.00       968

    accuracy                           1.00      4009
   macro avg       1.00      1.00      1.00      4009
weighted avg       1.00      1.00      1.00      4009



0.9992516837116487

In [29]:
from sklearn.metrics import classification_report
y_predRF = spamRF.predict(X_test)
print(classification_report(y_test, y_predRF))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1319
           1       0.95      0.94      0.95       400

    accuracy                           0.98      1719
   macro avg       0.97      0.97      0.97      1719
weighted avg       0.98      0.98      0.98      1719



In [30]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_predRF)
metrics.auc(fpr, tpr)

0.965297573919636

Which model had the best testing set performance, in terms of accuracy and AUC?

Random Forest is the best algorithm